In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df=pd.read_csv("/content/drive/MyDrive/EDA_DPA/Features/final_df.csv")

In [ ]:
df.head()

,id,file_name,title,state,potential_death,false_alarm,label,mfcc,zero_features,rms_features,sc_features,sb_features,sco_features,poly_features,label_encoded
0,1,call_1.mp3,North Hollywood bank robbery,California,1.0,0,genuine,2.774348,0.092506,0.091813,1122.137194,677.490571,25.617403,5.594756e-08,0
1,2,call_2.mp3,Detroit Child’s 911 Call – audio,Michigan,1.0,0,genuine,-0.440783,0.054411,0.130535,964.729144,1204.438309,24.651852,8.763465e-08,0
2,8,call_8.mp3,Girl’s Murder 911 Call,Florida,1.0,0,genuine,-0.309315,0.096928,0.104655,1355.358563,998.274181,15.986540,8.346468e-08,0
3,9,call_9.mp3,‘Shoot Her?’ 911 call,Texas,0.0,1,prank,1.410773,0.133110,0.028446,1532.378746,973.842686,17.838456,1.977618e-08,1
4,10,call_10.mp3,Snowstorm 911 Call,Nebraska,1.0,0,genuine,-3.020124,0.099188,0.081536,1110.698747,655.058693,23.605374,5.963549e-08,0


In [ ]:
columns=['id','file_name','title','state','potential_death','false_alarm']

In [ ]:
df=df[columns]

In [ ]:
df=df.set_index('id')

In [ ]:
df.head()

,file_name,title,state,potential_death,false_alarm
id,,,,,
1,call_1.mp3,North Hollywood bank robbery,California,1.0,0
2,call_2.mp3,Detroit Child’s 911 Call – audio,Michigan,1.0,0
8,call_8.mp3,Girl’s Murder 911 Call,Florida,1.0,0
9,call_9.mp3,‘Shoot Her?’ 911 call,Texas,0.0,1
10,call_10.mp3,Snowstorm 911 Call,Nebraska,1.0,0


In [ ]:
import pandas as pd
import gensim
from gensim import corpora
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import string

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
def preprocess_text(text):
    # Tokenize the text
    tokens = text.lower().split()
    # Remove punctuation and numbers
    tokens = [word for word in tokens if word.isalpha()]
    # Remove stopwords
    stop = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop]
    # Lemmatize words
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return tokens

# Apply the preprocessing to your DataFrame
df['title_tokens'] = df['title'].apply(preprocess_text)


In [ ]:
df.head()

,id,file_name,title,state,potential_death,false_alarm,label,mfcc,zero_features,rms_features,sc_features,sb_features,sco_features,poly_features,label_encoded,title_tokens
0,1,call_1.mp3,North Hollywood bank robbery,California,1.0,0,genuine,2.774348,0.092506,0.091813,1122.137194,677.490571,25.617403,5.594756e-08,0,"[north, hollywood, bank, robbery]"
1,2,call_2.mp3,Detroit Child’s 911 Call – audio,Michigan,1.0,0,genuine,-0.440783,0.054411,0.130535,964.729144,1204.438309,24.651852,8.763465e-08,0,"[detroit, call, audio]"
2,8,call_8.mp3,Girl’s Murder 911 Call,Florida,1.0,0,genuine,-0.309315,0.096928,0.104655,1355.358563,998.274181,15.986540,8.346468e-08,0,"[murder, call]"
3,9,call_9.mp3,‘Shoot Her?’ 911 call,Texas,0.0,1,prank,1.410773,0.133110,0.028446,1532.378746,973.842686,17.838456,1.977618e-08,1,[call]
4,10,call_10.mp3,Snowstorm 911 Call,Nebraska,1.0,0,genuine,-3.020124,0.099188,0.081536,1110.698747,655.058693,23.605374,5.963549e-08,0,"[snowstorm, call]"


In [ ]:
# Create a dictionary
dictionary = corpora.Dictionary(df['title_tokens'])

# Create a document-term matrix
doc_term_matrix = [dictionary.doc2bow(tokens) for tokens in df['title_tokens']]

In [ ]:
# Specify the number of topics you want to discover
num_topics = 7

# Create the LDA model
lda_model = gensim.models.LdaModel(
    doc_term_matrix,
    num_topics=num_topics,
    id2word=dictionary,
    passes=15
)

# Print the topics and their top words
for topic in lda_model.print_topics():
    print(topic)


(0, '0.094*"shooting" + 0.082*"stabbing" + 0.059*"invasion" + 0.059*"home" + 0.040*"drowning" + 0.039*"school" + 0.036*"victim" + 0.032*"accidental" + 0.028*"death" + 0.024*"kidnap"')
(1, '0.099*"shot" + 0.078*"cpr" + 0.051*"rescue" + 0.051*"call" + 0.041*"fire" + 0.028*"burglar" + 0.023*"hostage" + 0.022*"victim" + 0.019*"intruder" + 0.019*"closet"')
(2, '0.335*"murder" + 0.152*"call" + 0.027*"kidnapping" + 0.024*"multiple" + 0.024*"pursuit" + 0.018*"death" + 0.018*"double" + 0.015*"confession" + 0.012*"duo" + 0.012*"caller"')
(3, '0.097*"baby" + 0.092*"attack" + 0.090*"car" + 0.039*"delivery" + 0.034*"accident" + 0.030*"woman" + 0.015*"heart" + 0.015*"bear" + 0.012*"victim" + 0.010*"stabbed"')
(4, '0.157*"child" + 0.065*"robbery" + 0.058*"missing" + 0.041*"bank" + 0.035*"train" + 0.018*"murdered" + 0.018*"mother" + 0.018*"overdose" + 0.012*"choking" + 0.012*"abduction"')
(5, '0.273*"shooting" + 0.099*"officer" + 0.079*"crash" + 0.038*"plane" + 0.014*"found" + 0.014*"mall" + 0.011*"de

In [ ]:
df.head()

,id,file_name,title,state,potential_death,false_alarm,label,mfcc,zero_features,rms_features,sc_features,sb_features,sco_features,poly_features,label_encoded,title_tokens
0,1,call_1.mp3,North Hollywood bank robbery,California,1.0,0,genuine,2.774348,0.092506,0.091813,1122.137194,677.490571,25.617403,5.594756e-08,0,"[north, hollywood, bank, robbery]"
1,2,call_2.mp3,Detroit Child’s 911 Call – audio,Michigan,1.0,0,genuine,-0.440783,0.054411,0.130535,964.729144,1204.438309,24.651852,8.763465e-08,0,"[detroit, call, audio]"
2,8,call_8.mp3,Girl’s Murder 911 Call,Florida,1.0,0,genuine,-0.309315,0.096928,0.104655,1355.358563,998.274181,15.986540,8.346468e-08,0,"[murder, call]"
3,9,call_9.mp3,‘Shoot Her?’ 911 call,Texas,0.0,1,prank,1.410773,0.133110,0.028446,1532.378746,973.842686,17.838456,1.977618e-08,1,[call]
4,10,call_10.mp3,Snowstorm 911 Call,Nebraska,1.0,0,genuine,-3.020124,0.099188,0.081536,1110.698747,655.058693,23.605374,5.963549e-08,0,"[snowstorm, call]"


In [ ]:
df['title_tokens'].value_counts()

[]                           38
[murder]                     35
[shooting]                   22
[murder, call]               21
[officer, shooting]          14
                             ..
[carbon, monoxide, death]     1
[family, murder]              1
[resident, shot, police]      1
[heart, attack, cpr]          1
[moose, child]                1
Name: title_tokens, Length: 424, dtype: int64

In [ ]:
new_df=pd.read_csv("/content/drive/MyDrive/EDA_DPA/911_metadata.csv")

In [ ]:
new_df.head()

,id,event_id,link,title,date,state,deaths,potential_death,false_alarm,description,deaths_binary,break,filename
0,0,2,https://web.archive.org/web/20150417085342/htt...,Detroit Child’s 911 Call – audio,2/06,Michigan,1.0,1.0,0.0,– In Feb. 2006 5 year-old Robert Turner called...,1,13.0,911_first6sec/call_2_0.wav
1,1,8,https://web.archive.org/web/20150417085342/htt...,Girl’s Murder 911 Call,3/05,Florida,2.0,1.0,0.0,– the 911 call of a lifetime. Volusia County (...,1,3.0,911_first6sec/call_8_0.wav
2,2,9,https://web.archive.org/web/20150417085342/htt...,‘Shoot Her?’ 911 call,4/05,Texas,0.0,0.0,1.0,– caller reports her daughter is creating a di...,0,2.0,911_first6sec/call_9_0.wav
3,3,10,https://web.archive.org/web/20150417085342/htt...,Snowstorm 911 Call,1/05,Nebraska,2.0,1.0,0.0,– a couple under the influence of drugs dialed...,1,0.0,911_first6sec/call_10_0.wav
4,4,11,https://web.archive.org/web/20150417085342/htt...,911 Call for a Date,1/06,Oregon,0.0,0.0,1.0,– Lorna Jeanne Dudash in Aloha (Ore.) dialed 9...,0,2.0,911_first6sec/call_11_0.wav


In [ ]:
columns=['event_id','description']

In [ ]:
new_df=new_df[columns]

In [ ]:
new_df.head()

,event_id,description
0,2,– In Feb. 2006 5 year-old Robert Turner called...
1,8,– the 911 call of a lifetime. Volusia County (...
2,9,– caller reports her daughter is creating a di...
3,10,– a couple under the influence of drugs dialed...
4,11,– Lorna Jeanne Dudash in Aloha (Ore.) dialed 9...


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

In [ ]:
def lemmatize_text(text):
    if isinstance(text, str):
        lemmatizer = WordNetLemmatizer()
        tokens = text.split()
        lemmatized_tokens = [lemmatizer.lemmatize(token, wordnet.VERB) for token in tokens]
        return ' '.join(lemmatized_tokens)
    else:
        return ""  # Replace missing values with an empty string

# Preprocess the text
new_df['description'] = new_df['description'].apply(lemmatize_text)

In [ ]:
new_df.head()

,event_id,description
0,2,– In Feb. 2006 5 year-old Robert Turner call t...
1,8,– the 911 call of a lifetime. Volusia County (...
2,9,– caller report her daughter be create a distu...
3,10,– a couple under the influence of drug dial 91...
4,11,– Lorna Jeanne Dudash in Aloha (Ore.) dial 911...


In [ ]:
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(new_df['description'])

In [ ]:
# Apply LDA Topic Modeling
num_topics = 5  # You can adjust this based on your requirements
lda = LatentDirichletAllocation(n_components=num_topics, random_state=42)
lda.fit(tfidf_matrix)

LatentDirichletAllocation(n_components=5, random_state=42)

In [ ]:
# Get the top words for each topic
def get_top_words(model, vectorizer, n_words=10):
    words = vectorizer.get_feature_names_out()
    top_words = []
    for topic in model.components_:
        top_words_idx = topic.argsort()[:-n_words - 1:-1]
        top_words.append([words[i] for i in top_words_idx])
    return top_words

In [ ]:
top_words = get_top_words(lda, tfidf_vectorizer)

# Display the top words for each topic
for i, words in enumerate(top_words):
    print(f"Topic {i + 1}: {', '.join(words)}")

Topic 1: the, and, to, in, be, he, his, 911, her, dial
Topic 2: the, and, in, to, be, her, of, 911, call, an
Topic 3: the, 911, to, in, county, be, cause, fire, tex, on
Topic 4: the, and, in, to, be, his, shoot, man, 911, dial
Topic 5: the, and, in, be, to, 911, of, for, man, dial


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

# Define custom stopwords
custom_stopwords = ["the", "and", "to", "in", "be", "911"]

# TF-IDF Vectorization with custom stopwords
tfidf_vectorizer = TfidfVectorizer(stop_words=custom_stopwords, min_df=2)
tfidf_matrix = tfidf_vectorizer.fit_transform(new_df['description'])

# Increase the number of topics
num_topics = 10  # Adjust as needed
lda = LatentDirichletAllocation(n_components=num_topics, random_state=42)
lda.fit(tfidf_matrix)

# Display the top words for each topic
top_words = get_top_words(lda, tfidf_vectorizer)

for i, words in enumerate(top_words):
    print(f"Topic {i + 1}: {', '.join(words)}")

Topic 1: woman, call, she, her, who, from, by, at, stop, dial
Topic 2: non, collection, emergency, make, roberts, call, of, rifle, people, clackamas
Topic 3: store, incident, into, river, others, teen, grocery, raymond, rig, wisc
Topic 4: his, her, man, for, dial, he, an, shoot, later, of
Topic 5: of, call, woman, an, she, dispatcher, her, dial, police, fire
Topic 6: small, over, injure, make, 15, 200, fertilizer, massive, scores, mclennan
Topic 7: shoot, kill, dial, of, his, police, he, two, man, then
Topic 8: city, attacks, panic, fighter, 11th, jet, remember, fly, jersey, despite
Topic 9: her, dial, have, for, woman, dispatcher, his, he, man, she
Topic 10: bed, anderson, gunpoint, willis, storm, surround, saw, spree, hoover, might


In [ ]:
import pandas as pd
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [ ]:
nltk.download('vader_lexicon')  # Download the VADER lexicon
analyzer = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [ ]:
def analyze_sentiment(text):
    sentiment_scores = analyzer.polarity_scores(text)
    if sentiment_scores['compound'] >= 0.05:
        return "Positive"
    elif sentiment_scores['compound'] <= -0.05:
        return "Negative"
    else:
        return "Neutral"

In [ ]:
# Apply sentiment analysis to the 'description' column
new_df['sentiment'] = new_df['description'].apply(analyze_sentiment)

In [ ]:
new_df.head()

,event_id,description,sentiment
0,2,– In Feb. 2006 5 year-old Robert Turner call t...,Negative
1,8,– the 911 call of a lifetime. Volusia County (...,Negative
2,9,– caller report her daughter be create a distu...,Negative
3,10,– a couple under the influence of drug dial 91...,Neutral
4,11,– Lorna Jeanne Dudash in Aloha (Ore.) dial 911...,Negative


In [ ]:
new_df['sentiment'].value_counts()

Negative    580
Positive     79
Neutral      51
Name: sentiment, dtype: int64

In [ ]:
positive_descriptions = new_df[new_df['sentiment'] == "Positive"]

In [ ]:
positive_descriptions

,event_id,description,sentiment
10,18,– In Feb. 2007 a San Antonio (Tex.) family not...,Positive
14,27,"“I need a nanny,” the woman says. “Yeah, this ...",Positive
20,34,– In June 2007 a woman allegedly on drug drive...,Positive
53,69,Police officer and paramedics rush to the woma...,Positive
63,80,– In Sept. 2010 a Mesa (Ariz.) woman be splash...,Positive
...,...,...,...
673,704,– In 2007 a Kansas City (Mo.) landlord who be ...,Positive
676,707,– In June 2009 a woman walk her dog in a woode...,Positive
678,709,– In May 2010 the wife of an on-duty Clark Cou...,Positive
704,737,– In August 2010 a mother leave her twin 3 yea...,Positive


In [ ]:
nuetral_descriptions = new_df[new_df['sentiment'] == "Neutral"]

In [ ]:
nuetral_descriptions

,event_id,description,sentiment
3,10,– a couple under the influence of drug dial 91...,Neutral
12,20,"– On February 3, 2007 firefighters rescue 5 ch...",Neutral
15,28,– off-duty officer call to report he and his w...,Neutral
25,42,,Neutral
27,46,– A collection of non-emergency call make to 911.,Neutral
28,46,– A collection of non-emergency call make to 911.,Neutral
29,46,– A collection of non-emergency call make to 911.,Neutral
30,46,– A collection of non-emergency call make to 911.,Neutral
31,46,– A collection of non-emergency call make to 911.,Neutral
32,46,– A collection of non-emergency call make to 911.,Neutral


In [ ]:
pip install scikit-learn nltk

In [ ]:
import pandas as pd
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [ ]:
nltk.download('vader_lexicon')
analyzer = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [ ]:
new_df['compound_score'] = new_df['description'].apply(lambda x: analyzer.polarity_scores(x)['compound'])

In [ ]:
# Split the data into training and testing sets
X = new_df['description']
y = new_df['sentiment']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# TF-IDF Vectorization
tfidf_vectorizer = TfidfVectorizer(max_features=1000)  # Adjust the number of features as needed
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

In [ ]:

# Train a Random Forest classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train_tfidf, y_train)

RandomForestClassifier(random_state=42)

In [ ]:
# Make predictions on the test set
y_pred = clf.predict(X_test_tfidf)

In [ ]:
# Evaluate the model
print("Classification Report:")
print(classification_report(y_test, y_pred))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("Accuracy Score:", accuracy_score(y_test, y_pred))

Classification Report:
              precision    recall  f1-score   support

    Negative       0.87      0.99      0.93       115
     Neutral       0.88      0.58      0.70        12
    Positive       1.00      0.20      0.33        15

    accuracy                           0.87       142
   macro avg       0.92      0.59      0.65       142
weighted avg       0.88      0.87      0.84       142

Confusion Matrix:
[[114   1   0]
 [  5   7   0]
 [ 12   0   3]]
Accuracy Score: 0.8732394366197183
